In [1]:
#!pip3 install kerastuner

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


from sklearn.datasets import make_blobs
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import matthews_corrcoef,cohen_kappa_score,balanced_accuracy_score


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, LeakyReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.regularizers import l2


from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling,margin_sampling,entropy_sampling
from modAL.disagreement import KL_max_disagreement

from functools import partial


import warnings

warnings.filterwarnings('ignore')

2023-03-20 22:21:27.318489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data_source="./dataset/3901890_all.csv"
data= pd.read_csv(data_source)
data

,datetime,latitude,longitude,pressure,salinity,temperature,label
0,-1.873862,1.261588,0.939612,0.676593,0.967276,-0.896749,0
1,-1.873862,1.261588,0.939612,0.670563,0.967218,-0.896749,0
2,-1.873862,1.261588,0.939612,0.661519,0.967218,-0.895699,0
3,-1.873862,1.261588,0.939612,0.653336,0.967218,-0.894649,0
4,-1.873862,1.261588,0.939612,0.644723,0.967218,-0.894123,0
...,...,...,...,...,...,...,...
178316,1.563162,-1.925879,-1.540014,-1.242509,-1.141878,1.222459,0
178317,1.563162,-1.925879,-1.540014,-1.246816,-1.141065,1.221934,0
178318,1.563162,-1.925879,-1.540014,-1.251122,-1.141065,1.223509,0
178319,1.563162,-1.925879,-1.540014,-1.256291,-1.138048,1.225610,0


In [4]:
row,col=data.shape

In [5]:
#def compute ratio
def comp_ratio(data):
    instance = data[(data['label']==1)]
    rate=len(instance)/len(data)*100
    print(rate)
    return rate

In [6]:
dirty_ratio = comp_ratio(data)
dirty_ratio

29.364460719713325


29.364460719713325

# Splite Dataset

In [7]:
train_set=pd.read_csv('./randomtrain/3901890.csv')
test_set=pd.read_csv('./randomtest/3901890.csv')
len(train_set),len(test_set)

(124622, 53497)

In [8]:
#train_set2,test_set2=getLastSplit(data,0.3,dirty_ratio)

In [9]:
comp_ratio(train_set)
comp_ratio(test_set)

29.35918216687262
29.364263416640185


29.364263416640185

In [10]:
#this function use random grid search to find best parameters
def getPar(model,dist,data,niter):
    from sklearn.model_selection import RandomizedSearchCV
    x,y=data.shape
    clf = model

    param_dist = dist
    grid = RandomizedSearchCV(clf,param_dist,cv = 3,scoring = "balanced_accuracy",n_iter=niter,n_jobs = -1)

    #train
    grid.fit(data.iloc[:,0:y-1],data.iloc[:,y-1])
    #get best parameter
    print(grid.best_score_)
    return grid.best_params_

In [11]:
# metric
def computeMetric(y_tru,y_pre):
    acc = accuracy_score(y_tru,y_pre)
    pre=precision_score(y_tru,y_pre)
    recall=recall_score(y_tru,y_pre)
    cm=confusion_matrix(y_tru,y_pre)
    f1 = f1_score(y_tru,y_pre)
    mcc=matthews_corrcoef(y_tru, y_pre)
    kappa=cohen_kappa_score(y_tru, y_pre)
    bac=balanced_accuracy_score(y_tru,y_pre)
    print("acc:",acc)
    print("balanced acc:",bac)
    print("precision:",pre)
    print("recall:",recall)
    print("cm:",cm)
    print("f1:",f1)
    print("MCC:", mcc)
    print("Kappa:",kappa)
    
#     confusion matrix
#     cmap1 = sns.diverging_palette(260,-10,s=50, l=75, n=5, as_cmap=True)
#     plt.subplots(figsize=(12,8))
#     cf_matrix = confusion_matrix(y_tru, y_pre)
#     sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = cmap1, annot = True, annot_kws = {'size':15})
    
    return kappa

# splite train(pool), test, Labeled, Unlabeled

In [12]:
x,y=train_set.shape
#pool 
X_Pool = train_set.iloc[:,0:y-1].values
y_Pool = train_set.iloc[:,y-1].values

In [13]:
#test set
X_test=test_set.iloc[:,0:y-1]
y_true=test_set.iloc[:,y-1]
print(len(X_test),len(y_true))

53497 53497


In [14]:
# Set RNG seed for reproducibility.
RANDOM_STATE_SEED = 123
np.random.seed(RANDOM_STATE_SEED)

n_initial=100
N_QUERIES = 200

# Save initial training data

In [15]:
# X_initial,y_initial,X_re,y_re=initial_data(n_initial,X_Pool,y_Pool)
# df_X_i = pd.DataFrame(X_initial)
# df_y_i = pd.DataFrame(y_initial)
# df_X_r = pd.DataFrame(X_re)
# df_y_r = pd.DataFrame(y_re)


X_initial=pd.read_csv('./randomtrain/3901890_X_initial_1.csv').values
y_initial=pd.read_csv('./randomtrain/3901890_y_initial_1.csv').values.ravel()
X_re=pd.read_csv('./randomtrain/3901890_X_re_1.csv').values
y_re=pd.read_csv('./randomtrain/3901890_y_re_1.csv').values.ravel()
print(X_re.shape,X_initial.shape)
print(y_re.shape,y_initial.shape)

(124522, 6) (100, 6)
(124522,) (100,)


# molAL

In [16]:
def random_sampling(classifier, X_pool):
    n_samples = len(X_pool)
    query_idx = np.random.choice(range(n_samples))
    return query_idx, X_pool[query_idx]

In [17]:
def al(clf,strategy,X_L,y_L):
    learner = ActiveLearner(estimator=clf,
                            query_strategy=strategy,
                            X_training=X_L, y_training=y_L)
    return learner

In [18]:
def al_learn(model,sampling,X_re,y_re,X_t):
    X_L = X_initial.copy()
    y_L = y_initial.copy()
    X_U,y_U =X_re.copy(),y_re.copy()
    y_pre=model.predict(X_t)
    y_pre = (y_pre > 0.5).astype(int)
    unqueried_kappa=cohen_kappa_score(y_true, y_pre)
    unqueried_f1=f1_score(y_true,y_pre)
    print(unqueried_kappa,unqueried_f1)
    kappa_history = [unqueried_kappa]
    f1_history = [unqueried_f1]
    # the active learning loop
    n_queries = 40
    for idx in range(n_queries):
        query_idx = np.random.choice(range(len(y_U)),size=5)
        print(query_idx)
        X=X_U[query_idx]
        y=y_U[query_idx]
        print(idx+1,"query label --------------------->",y_re[query_idx])
        # update model on new data only with a smaller learning rate
        optimizer = RMSprop(learning_rate=0.0001)
        # compile the model
        model.compile(optimizer=optimizer, loss='binary_crossentropy')
        # fit the model on new data
        model.fit(X, y, epochs=100, verbose=0)
        # remove queried instance from pool
        X_U = np.delete(X_U, query_idx, axis=0)
        y_U = np.delete(y_U, query_idx, axis=0)
        y_pre=model.predict(X_t)
        y_pre = (y_pre > 0.5).astype(int)
        kappa=cohen_kappa_score(y_true, y_pre)
        f1=f1_score(y_true,y_pre)
        print(idx+1,"-------------------->",kappa,f1)
        # Recall precision F1
        kappa_history.append(kappa)
        f1_history.append(f1)
    #print(len(X_U))
    df_scores= pd.concat([pd.DataFrame(kappa_history,columns=['kappa']), 
                          pd.DataFrame(f1_history,columns=['f1'])],
                         axis=1)
    return df_scores


In [19]:
# model = Sequential()
# model.add(Dense(units=64, kernel_initializer='normal', activation='sigmoid', input_dim=X_Pool.shape[1]))
# model.add(Dropout(0.2))
# model.add(Dense(units=1, kernel_initializer='normal', activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])
# history = model.fit(X_Pool, y_Pool, batch_size=64, epochs=50, validation_data=(X_test, y_true))

In [20]:
def f1(y_true, y_pred):
    y_pred = tf.round(y_pred)
    return f1_score(y_true, y_pred)

In [21]:
# build function for the Keras' scikit-learn API
def create_baseline_model():
    model_b = Sequential()
    model_b.add(Dense(units=128, kernel_initializer='he_normal', input_dim=X_Pool.shape[1]))
    model_b.add(LeakyReLU())
    model_b.add(Dropout(0.5))
    model_b.add(Dense(units=64, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)))
    model_b.add(LeakyReLU())
    model_b.add(Dropout(0.3))
    model_b.add(Dense(units=32, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)))
    model_b.add(LeakyReLU())
    model_b.add(Dense(units=1, kernel_initializer='he_normal', activation='sigmoid'))
    optimizer = RMSprop(learning_rate=0.001)
    model_b.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_crossentropy'])

    return model_b


# create the classifier
model_b = create_baseline_model()

2023-03-20 22:21:35.341981: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
history = model_b.fit(X_initial, y_initial, batch_size=32, epochs=50, validation_data=(X_test, y_true))

Epoch 1/50
4/4 [==============================] - 3s 547ms/step - loss: 0.9388 - binary_crossentropy: 0.7491 - val_loss: 0.6970 - val_binary_crossentropy: 0.5078
Epoch 2/50
4/4 [==============================] - 1s 474ms/step - loss: 0.8793 - binary_crossentropy: 0.6904 - val_loss: 0.6331 - val_binary_crossentropy: 0.4447
Epoch 3/50
4/4 [==============================] - 1s 474ms/step - loss: 0.7713 - binary_crossentropy: 0.5830 - val_loss: 0.6096 - val_binary_crossentropy: 0.4217
Epoch 4/50
4/4 [==============================] - 1s 473ms/step - loss: 0.7461 - binary_crossentropy: 0.5582 - val_loss: 0.5885 - val_binary_crossentropy: 0.4010
Epoch 5/50
4/4 [==============================] - 1s 488ms/step - loss: 0.6990 - binary_crossentropy: 0.5117 - val_loss: 0.5781 - val_binary_crossentropy: 0.3911
Epoch 6/50
4/4 [==============================] - 2s 513ms/step - loss: 0.8184 - binary_crossentropy: 0.6315 - val_loss: 0.5731 - val_binary_crossentropy: 0.3866
Epoch 7/50
4/4 [============

In [23]:
y_pre=model_b.predict(X_test)
y_pred = (y_pre > 0.5).astype(int)
computeMetric(y_true,y_pred)

1672/1672 [==============================] - 2s 931us/step
acc: 0.9049479410060377
balanced acc: 0.8462026814323597
precision: 0.9623150565709312
recall: 0.7038640269908969
cm: [[37355   433]
 [ 4652 11057]]
f1: 0.8130445972278392
MCC: 0.7678754775155617
Kappa: 0.7513580236376436


0.7513580236376436

# RNN

In [24]:
# Reshape the input data to be 3D
print(X_Pool.shape,X_test.shape)
X_Pool_3 = X_Pool.reshape(X_Pool.shape[0], 1, X_Pool.shape[1])
X_initial_3=X_initial.reshape(X_initial.shape[0], 1, X_initial.shape[1])
X_re_3=X_re.reshape(X_re.shape[0], 1,X_re.shape[1])
X_test_3 = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

(124622, 6) (53497, 6)


In [25]:
print(y_Pool.shape,y_true.shape)

(124622,) (53497,)


In [26]:
# Build the RNN binary classification model
def create_RNN_model():
    model_R = Sequential()
    model_R.add(SimpleRNN(units=32, activation='relu', input_shape=(1, X_Pool.shape[1])))
    model_R.add(Dropout(0.2))
    model_R.add(Dense(units=1, activation='sigmoid'))
    model_R.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])
    return model_R

In [27]:
# Train the model
model_R = create_RNN_model()
history = model_R.fit(X_initial_3, y_initial, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
3/3 [==============================] - 1s 146ms/step - loss: 0.6098 - mse: 0.2124 - val_loss: 0.5954 - val_mse: 0.2043
Epoch 2/50
3/3 [==============================] - 0s 16ms/step - loss: 0.6021 - mse: 0.2068 - val_loss: 0.5800 - val_mse: 0.1975
Epoch 3/50
3/3 [==============================] - 0s 16ms/step - loss: 0.5961 - mse: 0.2054 - val_loss: 0.5659 - val_mse: 0.1914
Epoch 4/50
3/3 [==============================] - 0s 16ms/step - loss: 0.5987 - mse: 0.2075 - val_loss: 0.5537 - val_mse: 0.1862
Epoch 5/50
3/3 [==============================] - 0s 16ms/step - loss: 0.5547 - mse: 0.1874 - val_loss: 0.5421 - val_mse: 0.1814
Epoch 6/50
3/3 [==============================] - 0s 16ms/step - loss: 0.5792 - mse: 0.1979 - val_loss: 0.5312 - val_mse: 0.1769
Epoch 7/50
3/3 [==============================] - 0s 17ms/step - loss: 0.5675 - mse: 0.1932 - val_loss: 0.5212 - val_mse: 0.1728
Epoch 8/50
3/3 [==============================] - 0s 18ms/step - loss: 0.5375 - mse: 0.1815 - va

In [28]:
# Evaluate the model on the test set
y_pre=model_R.predict(X_test_3)
y_pred = (y_pre > 0.5).astype(int)

1672/1672 [==============================] - 2s 1ms/step


In [29]:
computeMetric(y_true,y_pred)

acc: 0.8656560180944726
balanced acc: 0.7831851684619446
precision: 0.9345298796655109
recall: 0.5833598574065822
cm: [[37146   642]
 [ 6545  9164]]
f1: 0.7183225553595924
MCC: 0.6666682705783421
Kappa: 0.6362132690106892


0.6362132690106892

## AL
### Baseline

In [30]:
"""
Training the ActiveLearner
"""
# initialize ActiveLearner
al_learn(model_b,"Random sampling",X_re,y_re,X_test)

1672/1672 [==============================] - 2s 1ms/step
0.7513580236376436 0.8130445972278392
[118270  15725  28030  17730  61404]
1 query label ---------------------> [1 0 0 0 0]
1672/1672 [==============================] - 2s 902us/step
1 --------------------> 0.7450981160424177 0.8144579959687063
[119906  63206  15377  73299  46203]
2 query label ---------------------> [0 1 1 0 0]
1672/1672 [==============================] - 2s 916us/step
2 --------------------> 0.7643487944849892 0.8299574810003683
[124473  89302  22241  65632  55409]
3 query label ---------------------> [0 0 0 0 0]
1672/1672 [==============================] - 2s 867us/step
3 --------------------> 0.7508231443471964 0.8138945233265721
[ 23166 118831 118857  71200  33710]
4 query label ---------------------> [1 1 0 1 0]
1672/1672 [==============================] - 2s 871us/step
4 --------------------> 0.7855102607188702 0.8419425117717337
[121056  65647 101785  17747  35662]
5 query label ---------------------> [1 

1672/1672 [==============================] - 2s 935us/step
40 --------------------> 0.7573730285118869 0.8258555133079848


,kappa,f1
0,0.751358,0.813045
1,0.745098,0.814458
2,0.764349,0.829957
3,0.750823,0.813895
4,0.785510,0.841943
5,0.725589,0.792039
6,0.802061,0.853556
7,0.770269,0.830722
8,0.725721,0.791784
9,0.745959,0.815224


In [31]:
# the final accuracy score
y_pre=model_b.predict(X_test)
y_pred = (y_pre > 0.5).astype(int)
computeMetric(y_true,y_pred)

1672/1672 [==============================] - 1s 879us/step
acc: 0.9015458810774436
balanced acc: 0.8704256628759259
precision: 0.8591772151898734
recall: 0.7950219619326501
cm: [[35741  2047]
 [ 3220 12489]]
f1: 0.8258555133079848
MCC: 0.7584811303725865
Kappa: 0.7573730285118869


0.7573730285118869

### RNN

In [32]:
"""
Training the ActiveLearner
"""
# initialize ActiveLearner
al_learn(model_R,"Random sampling",X_re_3,y_re,X_test_3)

1672/1672 [==============================] - 2s 928us/step
0.6362132690106892 0.7183225553595924
[ 18378 105548  41420  93849  17898]
1 query label ---------------------> [1 1 0 1 0]
1672/1672 [==============================] - 2s 899us/step
1 --------------------> 0.6691039392521744 0.7492236896255003
[111227  91633  36999  47885  34220]
2 query label ---------------------> [0 0 1 0 0]
1672/1672 [==============================] - 2s 972us/step
2 --------------------> 0.5906183499041513 0.6756367526642801
[66299 60673 71415 48809  1230]
3 query label ---------------------> [0 0 1 1 1]
1672/1672 [==============================] - 2s 994us/step
3 --------------------> 0.6656268285385578 0.7451812019540274
[101816  47575  37439  74209  18947]
4 query label ---------------------> [0 0 1 1 1]
1672/1672 [==============================] - 2s 986us/step
4 --------------------> 0.6615327080585931 0.7380374344114653
[  8209  46168  19671  78021 109153]
5 query label ---------------------> [0 0 0

1672/1672 [==============================] - 2s 955us/step
40 --------------------> 0.7141885664552268 0.7879371877230551


,kappa,f1
0,0.636213,0.718323
1,0.669104,0.749224
2,0.590618,0.675637
3,0.665627,0.745181
4,0.661533,0.738037
5,0.691780,0.763824
6,0.638352,0.717859
7,0.702070,0.772451
8,0.723076,0.790356
9,0.676635,0.750997


## random sampling

In [33]:
sampling=random_sampling

In [34]:
metric04 = al_learn(model_b,sampling,X_initial,y_initial,X_re,y_re)

TypeError: al_learn() takes 5 positional arguments but 6 were given

## Uncertainty

In [ ]:
sampling=uncertainty_sampling

In [ ]:
metric14 = al_learn(clf4,sampling,X_initial,y_initial,X_re,y_re)

## entropy

In [ ]:
sampling=entropy_sampling

In [ ]:
metric24 = al_learn(clf4,sampling,X_initial,y_initial,X_re,y_re)

## margin

In [ ]:
sampling=margin_sampling

In [ ]:
metric34 = al_learn(clf4,sampling,X_initial,y_initial,X_re,y_re)

# visual


In [ ]:
#save
def con_format(data):
    metrics_arr=np.transpose(metrics)# array T
    df = pd.DataFrame(metrics_arr, columns=['Random_Knn', 'Random_Lighgbm',"Random_GradientBoosting","Random_Catboost",
                                   'Uncertainty_Knn', 'Uncertainty_Lighgbm',"Uncertainty_GradientBoosting","Uncertainty_Catboost",
                                   'Entropy_Knn', 'Entropy_Lighgbm',"Entropy_GradientBoosting","Entropy_Catboost",
                                    'Margin_Knn', 'Margin_Lighgbm',"Margin_GradientBoosting","Margin_Catboost",
                                   ])
    return df
    

In [ ]:
metrics1= list(zip(metric01.iloc[:,1].values,metric02.iloc[:,1].values,metric03.iloc[:,1].values,metric04.iloc[:,1].values,
        metric11.iloc[:,1].values,metric12.iloc[:,1].values,metric13.iloc[:,1].values,metric14.iloc[:,1].values,
        metric21.iloc[:,1].values,metric22.iloc[:,1].values,metric23.iloc[:,1].values,metric24.iloc[:,1].values,
        metric31.iloc[:,1].values,metric32.iloc[:,1].values,metric33.iloc[:,1].values,metric34.iloc[:,1].values))


In [ ]:
df=con_format(metrics)
df

In [ ]:
# df.to_csv("./result/random_f1_3901890.csv")